# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_load = "../WeatherPy/weather_df.csv"
weather_data = pd.read_csv(weather_csv_load)
cleanedNA_weather_data = weather_data.dropna()
cleanedNA_weather_data

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Wind Speed (mph),Max Temp (F),Humidity (%)
0,0,georgetown,MY,2022-04-24 12:06:44,5.4112,100.3354,20.0,2.06,87.998,74.0
1,1,port elizabeth,ZA,2022-04-24 12:07:46,-33.9180,25.5701,14.0,3.58,60.368,97.0
2,2,gander,CA,2022-04-24 12:05:27,48.9682,-54.5906,100.0,9.26,32.432,100.0
3,3,avera,US,2022-04-24 12:07:48,33.1940,-82.5271,1.0,3.54,66.524,80.0
4,4,camana,PE,2022-04-24 12:07:50,-16.6228,-72.7111,25.0,2.89,67.820,87.0
...,...,...,...,...,...,...,...,...,...,...
633,633,beauvais,FR,2022-04-24 12:21:44,49.4167,2.0000,0.0,3.09,47.894,92.0
634,634,sisimiut,GL,2022-04-24 12:21:46,66.9395,-53.6735,100.0,2.38,28.436,96.0
635,635,baghdad,IQ,2022-04-24 12:21:47,33.3406,44.4009,100.0,3.09,77.180,23.0
636,636,mega,ET,2022-04-24 12:21:48,4.0500,38.3000,82.0,2.64,61.106,83.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Locations & weights 
locations = cleanedNA_weather_data[["Latitude", "Longitude"]]

humidity = cleanedNA_weather_data["Humidity (%)"].astype(float)

In [4]:
map = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating=False,max_intensity=100,point_radius=2)
map.add_layer(heatmap_layer)
map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Filter vacation with zero cloudiness
vacation_no_cloud = cleanedNA_weather_data[cleanedNA_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Wind Speed (mph),Max Temp (F),Humidity (%)
0,284,birin,NP,2022-04-24 12:13:59,26.8865,88.0718,0.0,2.39,72.788,52.0
1,406,ingham,US,2022-04-24 12:16:42,42.6000,-84.3833,0.0,3.60,72.896,59.0
2,447,guaymas,MX,2022-04-24 12:16:17,27.9333,-110.9000,0.0,3.56,70.160,40.0
3,511,gazojak,TM,2022-04-24 12:19:03,41.1875,61.4036,0.0,1.78,70.466,43.0
4,519,korla,CN,2022-04-24 12:19:14,41.7597,86.1469,0.0,1.93,76.028,17.0


In [7]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Wind Speed (mph),Max Temp (F),Humidity (%),Nearest Hotel
0,284,birin,NP,2022-04-24 12:13:59,26.8865,88.0718,0.0,2.39,72.788,52.0,Sangam hotel and lodge
1,406,ingham,US,2022-04-24 12:16:42,42.6000,-84.3833,0.0,3.60,72.896,59.0,Sunset Mobile Home Park
2,447,guaymas,MX,2022-04-24 12:16:17,27.9333,-110.9000,0.0,3.56,70.160,40.0,"Holiday Inn Express Guaymas, an IHG Hotel"
3,511,gazojak,TM,2022-04-24 12:19:03,41.1875,61.4036,0.0,1.78,70.466,43.0,Nearest hotel not found
4,519,korla,CN,2022-04-24 12:19:14,41.7597,86.1469,0.0,1.93,76.028,17.0,Gulisitan Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))